In [1]:
!pip install -q kaggle

from google.colab import files
files.upload()  # Upload the kaggle.json file

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sowaadrahman","key":"ce2fab7c1ebaf7fc9160542dc7722a29"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d markdaniellampa/fish-dataset


Dataset URL: https://www.kaggle.com/datasets/markdaniellampa/fish-dataset
License(s): Community Data License Agreement - Sharing - Version 1.0
 99% 1.57G/1.58G [00:20<00:00, 165MB/s]
100% 1.58G/1.58G [00:20<00:00, 81.6MB/s]


In [4]:
import os
import zipfile


zip_file_path = "fish-dataset.zip"
# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('dataset')

print("Extraction complete.")


if os.path.exists(zip_file_path):
    os.remove(zip_file_path)
    print(f"{zip_file_path} deleted.")
else:
    print(f"{zip_file_path} not found.")


Extraction complete.
fish-dataset.zip deleted.


**DATA PREPROCESSING**

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths for the train, validation, and test datasets
train_dir = '/content/dataset/FishImgDataset/train'
val_dir = '/content/dataset/FishImgDataset/val'
test_dir = '/content/dataset/FishImgDataset/test'


train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

val_test_datagen = ImageDataGenerator(rescale=1./255)

# Training data generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

# Validation data generator
val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

# Test data generator
test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')


Found 8791 images belonging to 31 classes.
Found 2751 images belonging to 31 classes.
Found 1760 images belonging to 31 classes.


**CNN+LSTM HYBRID MODEL**

In [6]:
from tensorflow.keras import layers, models


model = models.Sequential()

# First Convolutional Layer
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))

# Second Convolutional Layer
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Third Convolutional Layer
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Flatten the output from CNN
model.add(layers.Flatten())
print("Output shape after flattening: ", model.output_shape)


model.add(layers.Reshape((-1, model.output_shape[1])))

# Add LSTM layer
model.add(layers.LSTM(64, return_sequences=False))

# Fully connected layer
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(train_generator.num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


Output shape after flattening:  (None, 36992)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 34, 34, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 36992)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 1, 36992)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 64)                  │       9,486,592 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 31)                  │           3,999 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,592,159 (36.59 MB)

 Trainable params: 9,592,159 (36.59 MB)

 Non-trainable params: 0 (0.00 B)

**EPOCHS**

In [ ]:

history = model.fit(
    train_generator,
    epochs=15,
    validation_data=val_generator)


Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


275/275 ━━━━━━━━━━━━━━━━━━━━ 612s 2s/step - accuracy: 0.1346 - loss: 3.2569 - val_accuracy: 0.2116 - val_loss: 2.8774
Epoch 2/15
275/275 ━━━━━━━━━━━━━━━━━━━━ 613s 2s/step - accuracy: 0.2439 - loss: 2.7321 - val_accuracy: 0.3297 - val_loss: 2.3603
Epoch 3/15
275/275 ━━━━━━━━━━━━━━━━━━━━ 602s 2s/step - accuracy: 0.3303 - loss: 2.3995 - val_accuracy: 0.3933 - val_loss: 2.1422
Epoch 4/15
275/275 ━━━━━━━━━━━━━━━━━━━━ 599s 2s/step - accuracy: 0.3779 - loss: 2.1803 - val_accuracy: 0.4457 - val_loss: 1.9334
Epoch 5/15
275/275 ━━━━━━━━━━━━━━━━━━━━ 602s 2s/step - accuracy: 0.4513 - loss: 1.9289 - val_accuracy: 0.4762 - val_loss: 1.8373
Epoch 6/15
275/275 ━━━━━━━━━━━━━━━━━━━━ 621s 2s/step - accuracy: 0.4810 - loss: 1.7956 - val_accuracy: 0.5387 - val_loss: 1.6152
Epoch 7/15
214/275 ━━━━━━━━━━━━━━━━━━━━ 1:59 2s/step - accuracy: 0.5468 - loss: 1.5670

In [ ]:

test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc:.4f}')


**VIZUALIZATION**

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(12, 8))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()
